In [1]:
import torch
import numpy as np
from utils.generate_csbm import *
from utils.train import *
from utils.utils import *
from models.models import *
from models.setup import *
import networkx as nx
import pickle

In [2]:
for ith in range(20):
    file_to_open= open("experiments/csbm_SNC/0.88/data/"+"csbm_"+str(ith)+".pickle", "rb")
    csbm = pickle.load(file_to_open)
    file_to_open.close()
    train_ids = np.load("experiments/csbm_SNC/0.88/data/train_ids.npy")
    val_ids = np.load("experiments/csbm_SNC/0.88/data/val_ids.npy")
    test_ids = np.load("experiments/csbm_SNC/0.88/data/test_ids.npy")
    train_G = nx.from_numpy_matrix(csbm.A).subgraph(train_ids)

In [3]:
def exp_csbm(ith, method, folder_path, csbm, train_ids, val_ids, test_ids, 
             num_communication=100, batch_size=1,
             learning_rate=0.01, I=10, gradient=True, noise=False,
             Print=False, print_time=1,
             resume=False,
             bias=False,
             save=True):
    

    torch.manual_seed(0)
    init_mlp = MLP(csbm.Xs[0].shape[1], 64, 2, bias)
        
    if (gradient == False):
        grad = "no_grad/"
    elif (gradient == True and noise==False):
        grad = "biased_grad/"
    elif (gradient == True and noise==True):
        grad = "noisy_grad/"
      
    A_tilde = calculate_Atilde(csbm.A, K=10, alpha=0.9)
    server = set_up_NC(csbm.Xs, csbm.ys, init_mlp, A_tilde, train_ids, val_ids, test_ids)
   
        
    tl, ta, vl, va = train_NC(server, num_communication, batch_size, learning_rate, I,
                              gradient, noise, 
                              Print, print_time)
    if save:    
        np.save(folder_path + method + grad + "I" + str(I) + "/tl_" + str(ith), tl)
        np.save(folder_path + method + grad + "I" + str(I) + "/ta_" + str(ith), ta)
        np.save(folder_path + method + grad + "I" + str(I) + "/vl_" + str(ith), vl)
        np.save(folder_path + method + grad + "I" + str(I) + "/va_" + str(ith), va)

        PATH = folder_path + method + grad + "I" + str(I) + "/model_" + str(ith)
        torch.save({
                'best_model_state_dict': server.best_cmodel.state_dict(),
                'learning_rate': learning_rate,
                'test_acc': server.eval_test()[1],
                'model_state_dict': server.cmodel.state_dict(),
                'best_valloss': server.best_valloss,
                'best_valacc': server.best_valacc,
                }, PATH)

In [ ]:
csbm = cSBM(N=100, )

In [4]:
init_mlp = MLP(csbm.Xs.shape[2], 64, 2, bias=False)

In [5]:
A_tilde = calculate_Atilde(csbm.A, K=10, alpha=0.9)
server = set_up_NC(csbm.Xs, csbm.ys, init_mlp, A_tilde, train_ids, val_ids, test_ids)

In [6]:
tl, ta, vl, va = train_NC(server, num_communication=500, batch_size=50, learning_rate=0.05, I=10,
                          Print=True, print_time=10)

Communication: 1 Average train loss: 0.69622 Average train accuracy: 0.500 Average val loss: 0.68788 Average val accuracy: 0.550
Communication: 11 Average train loss: 0.69081 Average train accuracy: 0.559 Average val loss: 0.69211 Average val accuracy: 0.650
Communication: 21 Average train loss: 0.68887 Average train accuracy: 0.588 Average val loss: 0.69163 Average val accuracy: 0.750
Communication: 31 Average train loss: 0.68659 Average train accuracy: 0.629 Average val loss: 0.69102 Average val accuracy: 0.750
Communication: 41 Average train loss: 0.68382 Average train accuracy: 0.657 Average val loss: 0.69028 Average val accuracy: 0.750
Communication: 51 Average train loss: 0.68042 Average train accuracy: 0.674 Average val loss: 0.68938 Average val accuracy: 0.800
Communication: 61 Average train loss: 0.67623 Average train accuracy: 0.677 Average val loss: 0.68828 Average val accuracy: 0.800
Communication: 71 Average train loss: 0.67105 Average train accuracy: 0.700 Average val los

KeyboardInterrupt: 

In [8]:
server.eval_test()

(0.6805680934339762, 0.775)